In [ ]:
import json
import pandas as pd
from pathlib import Path

In [ ]:
verification_dir = Path('verification_assessment_data/')
dswx_verification_paths = list(verification_dir.glob('*/'))
dswx_ids = [path.name for path in dswx_verification_paths]
dswx_ids[:4]

In [ ]:
def read_one(dswx_id):
    path = verification_dir / dswx_id / f'requirement_verification_{dswx_id}.json'
    data = json.load(open(path))
    return data

In [ ]:
data = list(map(read_one, dswx_ids))

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
cols = ['surface_water','partial_surface_water']
df[cols + ['dswx_id']].groupby(cols).count()

In [ ]:
df.to_csv('verification_results.csv', index=False)